In [40]:
import pandas as pd
pd.set_option('display.max_columns', 500)


In [41]:
#this notebook is used for data exploration of Mechwarrior Online match data
#m is short for mech. m1 refers to the first mech in the afteraction picture
#The first 12 mechs are the winning team, the second 12 are the losing team
#Win refers to whether my team won the match
#r_dest is the number of enemy mechs destroyed
#b_dest is the number of my team's mechs destroyed
#Date uses YYYYMMDD format


In [42]:
#source of mech data: mwo.smurfy-net.de

mech_map_df = pd.read_csv("data/mech_datas.csv")
mech_map_df[mech_map_df.abbr.isin(["MAD", "RGH", "QKD", "TNS"])].head()
#name abbreviation
#tonnage
#class (can be mapped by tonnage)

,abbr,name,tons,faction,class
42,QKD,Quickdraw,60,IS,heavy
46,RGH,Roughneck,65,IS,heavy
53,MAD,Marauder,75,IS,heavy
55,TNS,Thanatos,75,IS,heavy


In [43]:
mwo_df = pd.read_csv("data/mwo_data.csv", sep=",")
mwo_df.head()

,date,time,r_dest,b_dest,win,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,m16,m17,m18,m19,m20,m21,m22,m23,m24,dmg_m1,dmg_m2,dmg_m3,dmg_m4,dmg_m5,dmg_m6,dmg_m7,dmg_m8,dmg_m9,dmg_m10,dmg_m11,dmg_m12,dmg_m13,dmg_m14,dmg_m15,dmg_m16,dmg_m17,dmg_m18,dmg_m19,dmg_m20,dmg_m21,dmg_m22,dmg_m23,dmg_m24,david,casie,jason
0,20171118,1124AM,7,12,1,MAD,BNC,EBJ,CP,KGC,NVA,BJ,MAD,MAL,ARC,KDK,ENF,ENF,ENF,ENF,SNV,QKD,QKD,TNS,TNS,WHM,NTG,RVN,GRF,602,331,248,222,1273,198,99,385,382,252,153,102,775,694,112,96,693,380,325,0,350,240,59,36,m11,m10,m8
1,20171118,1110AM,9,11,1,RGH,COU,NTG,NTG,ON1,KDK,LCT,UZL,SNV,TBR,DWF,PXH,UM,MAL,MAD,QKD,KGC,UM,EBJ,CPLT,MDD,SHC,GRF,NVA,732,314,347,100,750,738,247,89,409,436,384,34,409,340,270,78,1019,454,456,0,860,429,505,287,m3,m4,m1
2,20171118,1055AM,12,3,0,QKD,QKD,QKD,ENF,LBK,ASN,UM,UM,CP,SCR,VPR,LBK,GHR,CTF,TBR,WHM,SCR,LBK,MAD,WLF,RGH,NVA,EBJ,NTG,534,507,379,161,629,217,87,158,703,702,489,412,335,202,131,59,523,460,448,177,386,224,123,110,m24,m23,m21
3,20171118,1044AM,12,8,0,TNS,CPLT,SCR,SCR,BLR,WHM,MAD,KDK,SHD,TBR,CTF,WHM,RGH,ARC,NTG,HBK,MDD,GRF,WHM,KDK,KDK,SCR,HBR,UM,480,386,413,344,743,612,120,49,470,591,570,180,753,518,486,60,756,759,541,79,561,286,265,106,m3,m23,m1


In [44]:
#Set tonnage for all mechs in game
for i in range(1,25):
    col = "m"+str(i)
    mwo_df[col+"_tons"] = mwo_df[col].map(lambda x: mech_map_df.tons[mech_map_df.abbr==x].iloc[0])

In [90]:
#add descriptive stats to dataframe
win_tons_list = list(mwo_df.iloc[:,56:68]) #winning team mech tonnage
loss_tons_list = list(mwo_df.iloc[:, 68:80]) #losing team mech tons
mwo_df['win_total_tons'] = mwo_df[win_tons_list].sum(axis=1) #sum winning mech tonnage
mwo_df['loss_total_tons'] = mwo_df[loss_tons_list].sum(axis=1) #sum losing mech tonnage
#get tonnage difference, positive shows winning team was heavier
mwo_df['tonn_diff'] = mwo_df.apply(lambda x: x.win_total_tons - x.loss_total_tons, axis=1)  

In [93]:
mwo_df

,date,time,r_dest,b_dest,win,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,m16,m17,m18,m19,m20,m21,m22,m23,m24,dmg_m1,dmg_m2,dmg_m3,dmg_m4,dmg_m5,dmg_m6,dmg_m7,dmg_m8,dmg_m9,dmg_m10,dmg_m11,dmg_m12,dmg_m13,dmg_m14,dmg_m15,dmg_m16,dmg_m17,dmg_m18,dmg_m19,dmg_m20,dmg_m21,dmg_m22,dmg_m23,dmg_m24,david,casie,jason,m1_tons,m2_tons,m3_tons,m4_tons,m5_tons,m6_tons,m7_tons,m8_tons,m9_tons,m10_tons,m11_tons,m12_tons,m13_tons,m14_tons,m15_tons,m16_tons,m17_tons,m18_tons,m19_tons,m20_tons,m21_tons,m22_tons,m23_tons,m24_tons,red_total_tons,win_total_tons,loss_total_tons,tonn_diff
0,20171118,1124AM,7,12,1,MAD,BNC,EBJ,CP,KGC,NVA,BJ,MAD,MAL,ARC,KDK,ENF,ENF,ENF,ENF,SNV,QKD,QKD,TNS,TNS,WHM,NTG,RVN,GRF,602,331,248,222,1273,198,99,385,382,252,153,102,775,694,112,96,693,380,325,0,350,240,59,36,m11,m10,m8,75,95,65,90,100,50,45,75,90,70,100,50,50,50,50,90,60,60,75,75,70,75,35,55,NaN,905,745,160
1,20171118,1110AM,9,11,1,RGH,COU,NTG,NTG,ON1,KDK,LCT,UZL,SNV,TBR,DWF,PXH,UM,MAL,MAD,QKD,KGC,UM,EBJ,CPLT,MDD,SHC,GRF,NVA,732,314,347,100,750,738,247,89,409,436,384,34,409,340,270,78,1019,454,456,0,860,429,505,287,m3,m4,m1,65,35,75,75,75,100,20,50,90,75,100,45,30,90,75,60,100,30,65,65,60,45,55,50,NaN,805,725,80
2,20171118,1055AM,12,3,0,QKD,QKD,QKD,ENF,LBK,ASN,UM,UM,CP,SCR,VPR,LBK,GHR,CTF,TBR,WHM,SCR,LBK,MAD,WLF,RGH,NVA,EBJ,NTG,534,507,379,161,629,217,87,158,703,702,489,412,335,202,131,59,523,460,448,177,386,224,123,110,m24,m23,m21,60,60,60,50,65,40,30,30,90,55,40,65,70,70,75,70,55,65,75,35,65,50,65,75,NaN,645,770,-125
3,20171118,1044AM,12,8,0,TNS,CPLT,SCR,SCR,BLR,WHM,MAD,KDK,SHD,TBR,CTF,WHM,RGH,ARC,NTG,HBK,MDD,GRF,WHM,KDK,KDK,SCR,HBR,UM,480,386,413,344,743,612,120,49,470,591,570,180,753,518,486,60,756,759,541,79,561,286,265,106,m3,m23,m1,75,65,55,55,85,70,75,100,55,75,70,70,65,70,75,50,60,55,70,100,100,55,65,30,NaN,850,795,55


In [92]:
mwo_df[['tonn_diff', 'win']]

,tonn_diff,win
0,160,1
1,80,1
2,-125,0
3,55,0
